In [1]:
import tensorflow as tf
import tensorrt
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import glob

2023-05-11 22:22:01.002478: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-11 22:22:01.042856: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model = tf.keras.models.load_model('../saved_models/unet_incep/', compile = False)

2023-05-11 22:09:34.527310: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-11 22:09:34.548752: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-11 22:09:34.548872: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
converter = trt.TrtGraphConverterV2(
   input_saved_model_dir='../saved_models/unet_incep/',
   precision_mode=trt.TrtPrecisionMode.FP16
)

INFO:tensorflow:Linked TensorRT version: (8, 4, 3)
INFO:tensorflow:Loaded TensorRT version: (8, 6, 1)
INFO:tensorflow:Loaded TensorRT 8.6.1 and linked TensorFlow against TensorRT 8.4.3. This is supported because TensorRT minor/patch upgrades are backward compatible.


In [4]:
trt_func = converter.convert()
converter.summary()

2023-05-11 22:09:47.800616: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-11 22:09:47.800743: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-05-11 22:09:47.800830: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-11 22:09:47.801172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-11 22:09:47.801275: I tensorflow/compiler/xla/stream_executor/

INFO:tensorflow:Clearing prior device assignments in loaded saved model


2023-05-11 22:09:50.689837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-11 22:09:50.689973: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-05-11 22:09:50.690055: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-11 22:09:50.690370: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-11 22:09:50.690469: I tensorflow/compiler/xla/stream_executor/

TRTEngineOP Name                 Device        # Nodes # Inputs      # Outputs     Input DTypes       Output Dtypes      Input Shapes       Output Shapes     

----------------------------------------

TRTEngineOp_000_000              device:GPU:0  694     1             5             ['float32']        ['float32', 'f ... [[-1, -1, -1, 3]]  [[-1, -1, -1,  ...

	- AvgPool: 9x
	- ConcatV2: 11x
	- Const: 388x
	- Conv2D: 94x
	- FusedBatchNormV3: 94x
	- MaxPool: 4x
	- Relu: 94x

----------------------------------------

TRTEngineOp_000_001              device:GPU:0  18      2             1             ['float32', 'f ... ['float32']        [[-1, -1, -1,  ... [[-1, -1, -1,  ...

	- ConcatV2: 1x
	- Const: 11x
	- Conv2D: 2x
	- FusedBatchNormV3: 2x
	- Relu: 2x

----------------------------------------

TRTEngineOp_000_002              device:GPU:0  18      2             1             ['float32', 'f ... ['float32']        [[-1, -1, -1,  ... [[-1, -1, -1,  ...

	- ConcatV2: 1x
	- Const: 11x
	- Conv

In [5]:
def read_file(img):
    img = tf.io.read_file(img)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, size = (512, 512))
    img = img / 255
    return img

In [6]:
def dataset(generator):
    tensor = tf.data.Dataset.from_tensor_slices(generator)
    tensor = tensor.map(read_file)
    tensor = tensor.batch(32)
    return tensor

In [7]:
'../../files/train_v2/6384c3e78.jpg'

'../../files/train_v2/6384c3e78.jpg'

In [4]:
files = glob.glob('../../files/train_v2/*.jpg')
len(files)
files.remove('../../files/train_v2/6384c3e78.jpg')

In [5]:
len(files)

192555

In [10]:
tensor = dataset(files)

In [11]:
# This is not right thing to do
# converted_model = tf.keras.models.load_model('../saved_models/converted_model/')

In [6]:
import numpy as np

In [7]:
imgs = np.zeros(shape = (100, 512, 512, 3))
def read_and_batch_images(img):
    img = tf.io.read_file(img)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.image.resize(img, size = (512, 512))
    img = img / 255
    return img.numpy()

count = 0
for x in files[400:500]:
    i = read_and_batch_images(x)
    imgs[count] = i
    count += 1

In [8]:
imgs = imgs.astype(np.float32)

In [15]:
BATCH_SIZE = 32
def input_fn():
    batch_size = BATCH_SIZE
    x = imgs[0:batch_size, :]
    yield [x]

In [16]:
converter.build(input_fn=input_fn)

2023-05-11 22:10:00.824563: I tensorflow/compiler/tf2tensorrt/common/utils.cc:104] Linked TensorRT version: 8.4.3
2023-05-11 22:10:00.824657: I tensorflow/compiler/tf2tensorrt/common/utils.cc:106] Loaded TensorRT version: 8.6.1
2023-05-11 22:10:07.339787: I tensorflow/compiler/tf2tensorrt/convert/convert_nodes.cc:1330] [TF-TRT] Sparse compute capability: enabled.
2023-05-11 22:13:33.049577: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger TensorRT encountered issues when converting weights between types and that could affect accuracy.
2023-05-11 22:13:33.049607: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger Check verbose logs for the list of affected weights.
2023-05-11 22:13:33.049613: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger - 87 weights are affected by this issue: Detected subnormal FP16 values.
2023-05-11 22:13:33.049650: W tensorflow/compiler/tf2tensorrt/

In [17]:
converter.save('../saved_models/converted_model/')

INFO:tensorflow:Assets written to: ../saved_models/converted_model/assets


In [18]:
import time

In [19]:
start = time.time()
preds = trt_func(tf.convert_to_tensor((imgs[:32, :]), dtype = tf.float32))
end = time.time()
print(end - start)

2023-05-11 22:16:43.672202: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger TensorRT encountered issues when converting weights between types and that could affect accuracy.
2023-05-11 22:16:43.672232: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger Check verbose logs for the list of affected weights.
2023-05-11 22:16:43.672237: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger - 87 weights are affected by this issue: Detected subnormal FP16 values.
2023-05-11 22:16:43.672273: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger - 35 weights are affected by this issue: Detected values less than smallest positive FP16 subnormal value and converted them to the FP16 minimum subnormalized value.
2023-05-11 22:16:49.548258: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger TensorRT encountered issues wh

60.586119651794434


2023-05-11 22:17:13.552981: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger TensorRT encountered issues when converting weights between types and that could affect accuracy.
2023-05-11 22:17:13.553009: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger Check verbose logs for the list of affected weights.
2023-05-11 22:17:13.553014: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger - 1 weights are affected by this issue: Detected subnormal FP16 values.


In [22]:
start = time.time()
preds = trt_func(tf.convert_to_tensor((imgs[:32, :]), dtype = tf.float32))
end = time.time()
print(end - start)

0.02176833152770996


In [20]:
start = time.time()
preds = model.predict(imgs[:32, :], verbose = 1)
end = time.time()
print(end - start)

2023-05-11 22:17:15.628454: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8901
2023-05-11 22:17:17.019817: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 5s 5s/step
4.698083877563477


In [1]:
import tensorrt
import tensorflow as tf

2023-05-11 22:00:47.112138: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-11 22:00:47.151717: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def get_func_from_saved_model(saved_model_dir):
   saved_model_loaded = tf.saved_model.load(
       saved_model_dir, tags=[tag_constants.SERVING])
   graph_func = saved_model_loaded.signatures[
       signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
   return graph_func, saved_model_loaded

In [2]:
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import tag_constants

In [3]:
saved_model_loaded = tf.saved_model.load(
       '../saved_models/converted_model/', tags=[tag_constants.SERVING])
graph_func = saved_model_loaded.signatures[
       signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

2023-05-11 22:22:07.574288: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-11 22:22:07.595719: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-11 22:22:07.595841: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [13]:
# graph_func(tf.convert_to_tensor((imgs[32:64, :]), dtype = tf.float32))